In [5]:
# [3회차 기출유형 작업형2] : 여행 보험 패키지 상품 (데이터를 조금 어렵게 변경함)
# 여행 보험 패키지 상품을 구매할 확률 값을 구하시오
# 예측할 값(y): TravelInsurance (여행보험 패지지를 구매 했는지 여부 0:구매안함, 1:구매)
# 평가: roc-auc 평가지표
# 풀이

import pandas as pd

train = pd.read_csv('./data/t2-1-train.csv')
test = pd.read_csv('./data/t2-1-test.csv')

# print(train.head(3))
# print(train.info())
# print(train.isnull().sum()) # AnnualIncome(4)
# print(train.shape) # (1490, 10)

# 이진 분류 Regression?
from sklearn.models import Regression
from sklearn.preprocessing import roc_auc_val

# corr? 같은 열 빼고

model = Regression()
model.fit(train, train['TravelInserance'])

model.roc_auc_val()

model.predict(test)

model.to_csv('201734007.csv')

res = pd.read_csv('./201734007.csv')
print(res.head())

['Binarizer',
 'FunctionTransformer',
 'KBinsDiscretizer',
 'KernelCenterer',
 'LabelBinarizer',
 'LabelEncoder',
 'MaxAbsScaler',
 'MinMaxScaler',
 'MultiLabelBinarizer',
 'Normalizer',
 'OneHotEncoder',
 'OrdinalEncoder',
 'PolynomialFeatures',
 'PowerTransformer',
 'QuantileTransformer',
 'RobustScaler',
 'SplineTransformer',
 'StandardScaler',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_csr_polynomial_expansion',
 '_data',
 '_discretization',
 '_encoders',
 '_function_transformer',
 '_label',
 '_polynomial',
 'add_dummy_feature',
 'binarize',
 'label_binarize',
 'maxabs_scale',
 'minmax_scale',
 'normalize',
 'power_transform',
 'quantile_transform',
 'robust_scale',
 'scale']

In [23]:
# 해답

import pandas as pd

train = pd.read_csv('./data/t2-1-train.csv')
test = pd.read_csv('./data/t2-1-test.csv')

# print(train.shape, test.shape)

# display(train.describe(include='object'))
# display(test.describe(include='object'))

# print(train['Employment Type'].value_counts())
# print(test['Employment Type'].value_counts())

# 결측치 처리
train['AnnualIncome'] = train['AnnualIncome'].fillna(train['AnnualIncome'].mean())
test['AnnualIncome'] = test['AnnualIncome'].fillna(test['AnnualIncome'].mean())

target = train.pop('TravelInsurance')

df = pd.concat([train, test])

from sklearn.preprocessing import LabelEncoder

cols = df.select_dtypes(include='object').columns
le = LabelEncoder()

# print(df.head(3))
for col in cols:
    df[col] = le.fit_transform(df[col])
# print(df.head(3))

train = df[:train.shape[0]].copy()
test = df[train.shape[0]:].copy()

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

train['AnnualIncome'] = scaler.fit_transform(train[['AnnualIncome']])
test['AnnualIncome'] = scaler.fit_transform(test[['AnnualIncome']])

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=2022)

# 의사결정나무
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=2022)
model.fit(X_train, y_train)
pred = model.predict_proba(X_val)

from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_val, pred[:,1]))
# print(pred[:5])

# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=2022)
model.fit(X_train, y_train)
pred = model.predict_proba(X_val)
print(roc_auc_score(y_val, pred[:,1]))

import xgboost as xgb
model = xgb.XGBRFClassifier(random_state=2022)
model.fit(X_train, y_train)
pred = model.predict_proba(X_val)
print(roc_auc_score(y_val, pred[:,1]))

# 예측
model = RandomForestClassifier(random_state=2022)
model.fit(X_train, y_train)
pred = model.predict_proba(test)

submit = pd.DataFrame()
submit['id'] = test['id']
submit['TravelInsurance'] = pred[:,1]
print(submit.head())

submit.to_csv('2022_1.csv', index=False)
print(pd.read_csv('2022_1.csv'))

0.6987878787878787
0.7659090909090909
0.7587373737373737
   id  TravelInsurance
0   0             0.31
1   1             0.48
2   2             0.30
3   3             0.83
4   4             0.28
      id  TravelInsurance
0      0             0.31
1      1             0.48
2      2             0.30
3      3             0.83
4      4             0.28
..   ...              ...
492  492             0.13
493  493             0.96
494  494             0.16
495  495             0.73
496  496             0.92

[497 rows x 2 columns]


In [24]:
type(target)

pandas.core.series.Series